# Building a Spam Filter with Naive Bayes Algorithm

In this project, I will build a spam filter for SMS messages.

To classify messages as spam or non-spam, the computer will:
1. Learn how humans classify messages
2. Use that knowledge to estimate spam and non-spam probabilities for new messages
3. Classify a new message based on these probability values

Our first task is to 'teach' the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

Let's start by reading in the dataset.

In [12]:
import pandas as pd
messages = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
print(messages.shape)
num_spam = len(messages[messages['Label'] == 'spam'])
num_non_spam = len(messages[messages['Label'] == 'ham'])  # 'ham' means non-spam
print('\nSpam: ', num_spam, '\nPct Spam: ', round(100 * num_spam / len(messages), 2), '%', 
      '\n\nNon-spam: ', num_non_spam, '\nPct non-spam: ', round(100 * num_non_spam / len(messages), 2), '%')
messages.head()

(5572, 2)

Spam:  747 
Pct Spam:  13.41 % 

Non-spam:  4825 
Pct non-spam:  86.59 %


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Approx. 13% of messages are spam, and the remaining 87% are non-spam.

Before creating the spam filter, I will first design the test.

To test the spam filter, I will first split the dataset into two categories:
- a **training set**, which will be used to train the computer how to classify messages
- a **test set**, which will be used to test how good the spam filter is with classifying new messages

We will keep 80% of our dataset for training (4,458 messages), and 20% for testing (1,114 messages). Since the 1,114 messages in the test set were already classified by a human, we can compare the algorithm classification with the human classification to see how well it works. Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%.

I will start by creating the training and test datasets.

In [13]:
# Randomize entire dataset:
random = messages.sample(frac=1, random_state=1)

# Calculate index to split on
train_test_index = round(len(random) * .8)

# Create training and test data sets
training_set = random[:train_test_index].reset_index(drop=True)
test_set = random[train_test_index:].reset_index(drop=True)

print('Training set shape: ', training_set.shape, '\nTest set shape: ', test_set.shape)

# Find percentage of spam and ham in both training and test sets to ensure similar percentages to full dataset

train_n_spam = len(training_set[training_set['Label'] == 'spam'])
train_pct_spam = 100 * train_n_spam / len(training_set)

train_n_ham = len(training_set[training_set['Label'] == 'ham'])
train_pct_ham = 100 * train_n_ham / len(training_set)

test_n_spam = len(test_set[test_set['Label'] == 'spam'])
test_pct_spam = 100 * test_n_spam / len(test_set)

test_n_ham = len(test_set[test_set['Label'] == 'ham'])
test_pct_ham = 100 * test_n_ham / len(test_set)

print('\nTraining pct spam: {}%\nTraining pct ham: {}%\n\nTest pct spam: {}%\nTest pct ham: {}%'.format(
round(train_pct_spam, 2), round(train_pct_ham, 2), round(test_pct_spam, 2), round(test_pct_ham, 2)))

Training set shape:  (4458, 2) 
Test set shape:  (1114, 2)

Training pct spam: 13.46%
Training pct ham: 86.54%

Test pct spam: 13.2%
Test pct ham: 86.8%


Both of the training and test sets conntain approx. 13% spam messages and 87% ham messages, which is the same as the original dataset. Thus, our training and test datasets will work well.

The next step is to use the training set to teach the algorithm to classify new messages. 

When a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to these two equations:

P(Spam|w1,w2,...,wn) ∝ P(Spam)⋅∏P(wi|Spam)

P(Ham|w1,w2,...,wn) ∝ P(Ham)⋅∏P(wi|Ham)

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

P(wi|Spam) = (N(wi|Spam) + α) / (NSpam + α⋅NVocabulary)

P(wi|Ham) = (N(wi|Ham) + α) / (NHam + α⋅NVocabulary)

Let's also summarize what the terms in the equations above mean:

- N(wi|Spam) = the number of times the word wi occurs in spam messages
- N(wi|Ham) = the number of times the word wi occurs in non-spam messages
- NSpam = total number of words in spam messages
- NHam = total number of words in non-spam messages
- NVocabulary = total number of words in the vocabulary
- α = 1    (α is a smoothing parameter)

To make the calculations easier, we will replace the SMS column with columns for each word in all of the messages, whos values are the number of times that word appeared in the SMS message.

I will start this process by first removing the punctuation and bringing all words to lower case.

In [14]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ').str.lower()

I will now create a vocabulary for the messages in the training set, which is a set of all of the unique words in the messages.

In [15]:
training_set['SMS'] = training_set['SMS'].str.split() # split each of the messages into a list of words

vocabulary = []
for message in training_set['SMS']: # add each word to the vocabulary list
    for word in message:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary)) # transform to a set to remove duplicates, then back to a list
len(vocabulary)

7783

There are 7783 unique words in the vocabulary

Lets create a dictionary of each unique word and its count per sms, and then transform that into a dataframe.

In [16]:
# Initialize empry dictionary with each word having a 0 for each message in the training set
word_counts_per_message = {word: [0] * len(training_set['SMS']) for word in vocabulary}

# Iterate through each message, and get word counts
for index, message in enumerate(training_set['SMS']):
    for word in message:
        word_counts_per_message[word][index] += 1
        

In [18]:
word_counts = pd.DataFrame(word_counts_per_message)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [19]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


### Calculating constants

Now that we have our dataframe we want to work with, we'll begin by calculating P(Spam), P(Ham), NSpam, NHam, and NVocabulary.

NSpam is equal to the number of words in all the spam messages, not the number of spam messages or number of unique words in spam messages
NHam is equal to the number of words in all non-spam messages

We'll also use Laplace smoothing and set α = 1

In [22]:
# Isolate spam and ham messages
spam = training_set_clean[training_set_clean['Label'] == 'spam']
ham = training_set_clean[training_set_clean['Label'] == 'ham']

# Calculate p_spam and p_ham
p_spam = len(spam) / len(training_set_clean)
p_ham = len(ham) / len(training_set_clean)

# Calculate NSpam, NHam, and NVocabulary
n_words_per_spam_message = spam['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

n_words_per_ham_message = ham['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()
    
n_vocabulary = len(vocabulary)

alpha = 1

### Calculating Parameters

Our results will come from the equations:
- P(Spam|w1,w2,...,wn) ∝ P(Spam)⋅∏P(wi|Spam)
- P(Ham|w1,w2,...,wn) ∝ P(Ham)⋅∏P(wi|Ham)

where 
- P(wi|Spam) = (N(wi|Spam) + α) / (NSpam + α⋅NVocabulary)
- P(wi|Ham) = (N(wi|Ham) + α) / (NHam + α⋅NVocabulary)

Let's calculate the parameters P(wi|Spam) and P(wi|Ham):

In [24]:
# Initialize empty dictionaries
spam_param = {word: 0 for word in vocabulary}
ham_param = {word: 0 for word in vocabulary}

for word in vocabulary:
    n_word_given_spam = spam[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    spam_param[word] = p_word_given_spam
    
    n_word_given_ham = ham[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    ham_param[word] = p_word_given_ham


### Creating the spam filter

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter, which will be a function that:
- Takes in an input of a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn) and:
    - if P(Spam|w1, w2, ..., wn) < P(Ham|w1, w2, ..., wn), the message is classified as ham
    - if P(Spam|w1, w2, ..., wn) > P(Ham|w1, w2, ..., wn), the message is classified as spam
    - if P(Spam|w1, w2, ..., wn) = P(Ham|w1, w2, ..., wn), the algorithm may request human help

In [25]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in spam_param:
            p_spam_given_message *= spam_param[word]
        if word in ham_param:
            p_ham_given_message *= ham_param[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [26]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [27]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


The classify function appears to work well. We will now use it to test our set of 1,114 messages.

In [30]:
# Create new classify function that returns labels instead of printing them
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_param:
            p_spam_given_message *= spam_param[word]
        if word in ham_param:
            p_ham_given_message *= ham_param[word]
            
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    


In [34]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

test_set['correct'] = test_set['predicted'] == test_set['Label']
correct=test_set['correct'].sum()
total = len(test_set)        
percent_correct = 100 * correct / total

print('Correct: ', correct)
print('Incorrect', total - correct)
print('Percent correct: ', round(percent_correct, 2), '%')

Correct:  1100
Incorrect 14
Percent correct:  98.74 %


We accurately predicted 98% of the labels!

Let's look at the messages we didn't correctly predict:

In [35]:
test_set[test_set['correct'] == False]

,Label,SMS,predicted,correct
114,spam,Not heard from U4 a while. Call me now am here...,ham,False
135,spam,More people are dogging in your area now. Call...,ham,False
152,ham,Unlimited texts. Limited minutes.,spam,False
159,ham,26th OF JULY,spam,False
284,ham,Nokia phone is lovly..,spam,False
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,needs human classification,False
302,ham,No calls..messages..missed calls,spam,False
319,ham,We have sent JD for Customer Service cum Accou...,spam,False
504,spam,Oh my god! I've found your number again! I'm s...,ham,False
546,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham,False


- One of the predicted needed human classification.
- Many of the spam messages incorrectly predicted as ham sound like ham and I am unsurprised the computer predicted ham
- Many of the ham messages incorrectly predicted as spam contained words that sound sort of 'spammy'

Overall, it appears that our spam filter works very well.